# Step 1: Create industrial, domestic, and livestock demand
This script summarizes the PCR-GLOBWB NetCDF outputs for non-irrigation (livestock, domestic, industrical) demand and consumption by the Hydrobasin 6 catchments. We loop through the scen, convert the flux to volume, and take a clip of the timeseries data for each polygon. Data is saved by demand type, region and pfafid 


## Gross demand
gdomww = domesticGrossDemand  * area (million m3//month)
gindww = industryGrossDemand  * area (million m3//month)
glivww = livestockGrossDemand  * area (million m3//month)
 ## Gross consumption
gdomwn =  (domesticNetoDemand/ domesticGrossDemand) * gdomww
gindwn =  (industryNetoDemand/ industryGrossDemand) * gindww
glivwn = glivww

 ## Next steps
 - adjust to account for resample (5x5)

# Install Libraries

In [0]:
!pip install tqdm
!pip install rtree
!pip3 install numpy
!pip3 install pandas
!pip3 install scipy
!pip3 install geopandas
!pip3 install xarray
!pip3 install rasterio
!pip3 install rasterstats
!pip3 install rioxarray
!pip3 install netcdf4
!pip install psutil
!pip install dask
import psutil
import xarray
import rioxarray
import rasterio
import geopandas as gpd
import rasterstats as rstats
import netCDF4, os, subprocess, re, time, datetime, json
import numpy as np, pandas as pd
import netCDF4 as nc
from rasterio import Affine
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
import math
from tqdm import tqdm
import dask
import gc
from joblib import Parallel, delayed


Collecting tqdm
 Downloading tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
████▏ | 10 kB 28.3 MB/s eta 0:00:01 |████████▍ | 20 kB 18.1 MB/s eta 0:00:01 |████████████▌ | 30 kB 23.7 MB/s eta 0:00:01 |████████████████▊ | 40 kB 17.8 MB/s eta 0:00:01 |████████████████████▉ | 51 kB 15.8 MB/s eta 0:00:01 |█████████████████████████ | 61 kB 18.0 MB/s eta 0:00:01 |█████████████████████████████▎ | 71 kB 19.8 MB/s eta 0:00:01 |████████████████████████████████| 78 kB 5.4 MB/s 
Installing collected packages: tqdm
Successfully installed tqdm-4.64.1
 WARNING: You are using pip version 21.0.1; however, version 23.0 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Collecting rtree
 Downloading Rtree-1.0.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
▎ | 10 kB 29.3 MB/s eta 0:00:01 |▋ | 20 kB 35.6 MB/s eta 0:00:01 |█ | 30 kB 41.2 MB/s eta 0:00:01 |█▎ | 40 kB 19.5 MB/s eta 0:00:01 |█▋ | 51 kB 22.3 MB/s eta 0:00:01 |██ | 61 kB 24.8 MB/s eta 0:00:01 |██▎ | 71 kB 21.3 MB/s eta 0:00:01 |██▌ | 81 kB 23.2 MB/s eta 0:00:01 |██▉ | 92 kB 24.8 MB/s eta 0:00:01 |███▏ | 102 kB 25.3 MB/s eta 0:00:01 |███▌ | 112 kB 25.3 MB/s eta 0:00:01 |███▉ | 122 kB 25.3 MB/s eta 0:00:01 |████▏ | 133 kB 25.3 MB/s eta 0:00:01 |████▌ | 143 kB 25.3 MB/s eta 0:00:01 |████▊ | 153 kB 25.3 MB/s eta 0:00:01 |█████ | 163 kB 25.3 MB/s eta 0:00:01 |█████▍ | 174 kB 25.3 MB/s eta 0:00:01 |█████▊ | 184 kB 25.3 MB/s eta 0:00:01 |██████ | 194 kB 25.3 MB/s eta 0:00:01 |██████▍ | 204 kB 25.3 MB/s eta 0:00:01 |██████▊ | 215 kB 25.3 MB/s eta 0:00:01 |███████ | 225 kB 25.3 MB/s eta 0:00:01 |███████▎ | 235 kB 25.3 MB/s eta 0:00:01 |███████▋ | 245 kB 25.3 MB/s eta 0:00:01 |████████ | 256 kB 25.3 MB/s eta 0:00:01 |████████▎ | 266 kB 25.3 MB/s eta 0:00:01 |████████▋ | 276 kB 25.3 MB/s eta 0:00:01 |█████████ | 286 kB 25.3 MB/s eta 0:00:01 |█████████▏ | 296 kB 25.3 MB/s eta 0:00:01 |█████████▌ | 307 kB 25.3 MB/s eta 0:00:01 |█████████▉ | 317 kB 25.3 MB/s eta 0:00:01 |██████████▏ | 327 kB 25.3 MB/s eta 0:00:01 |██████████▌ | 337 kB 25.3 MB/s eta 0:00:01 |██████████▉ | 348 kB 25.3 MB/s eta 0:00:01 |███████████▏ | 358 kB 25.3 MB/s eta 0:00:01 |███████████▍ | 368 kB 25.3 MB/s eta 0:00:01 |███████████▊ | 378 kB 25.3 MB/s eta 0:00:01 |████████████ | 389 kB 25.3 MB/s eta 0:00:01 |████████████▍ | 399 kB 25.3 MB/s eta 0:00:01 |████████████▊ | 409 kB 25.3 MB/s eta 0:00:01 |█████████████ | 419 kB 25.3 MB/s eta 0:00:01 |█████████████▍ | 430 kB 25.3 MB/s eta 0:00:01 |█████████████▊ | 440 kB 25.3 MB/s eta 0:00:01 |██████████████ | 450 kB 25.3 MB/s eta 0:00:01 |██████████████▎ | 460 kB 25.3 MB/s eta 0:00:01 |██████████████▋ | 471 kB 25.3 MB/s eta 0:00:01 |███████████████ | 481 kB 25.3 MB/s eta 0:00:01 |███████████████▎ | 491 kB 25.3 MB/s eta 0:00:01 |███████████████▋ | 501 kB 25.3 MB/s eta 0:00:01 |████████████████ | 512 kB 25.3 MB/s eta 0:00:01 |████████████████▏ | 522 kB 25.3 MB/s eta 0:00:01 |████████████████▌ | 532 kB 25.3 MB/s eta 0:00:01 |████████████████▉ | 542 kB 25.3 MB/s eta 0:00:01 |█████████████████▏ | 552 kB 25.3 MB/s eta 0:00:01 |█████████████████▌ | 563 kB 25.3 MB/s eta 0:00:01 |█████████████████▉ | 573 kB 25.3 MB/s eta 0:00:01 |██████████████████▏ | 583 kB 25.3 MB/s eta 0:00:01 |██████████████████▍ | 593 kB 25.3 MB/s eta 0:00:01 |██████████████████▊ | 604 kB 25.3 MB/s eta 0:00:01 |███████████████████ | 614 kB 25.3 MB/s eta 0:00:01 |███████████████████▍ | 624 kB 25.3 MB/s eta 0:00:01 |███████████████████▊ | 634 kB 25.3 MB/s eta 0:00:01 |████████████████████ | 645 kB 25.3 MB/s eta 0:00:01 |████████████████████▍ | 655 kB 25.3 MB/s eta 0:00:01 |████████████████████▋ | 665 kB 25.3 MB/s eta 0:00:01 |█████████████████████ | 675 kB 25.3 MB/s eta 0:00:01 |█████████████████████▎ | 686 kB 25.3 MB/s eta 0:00:01 |█████████████████████▋ | 696 kB 25.3 MB/s eta 0:00:01 |██████████████████████ | 706 kB 25.3 MB/s eta 0:00:01 |███████████████████

--------------------------------------------------------------------------- 
 ImportError Traceback (most recent call last)
 <command-3196742707517586> in <module> 
 13 get_ipython ( ) . system ( 'pip install dask' ) 
 14 import psutil
 ---> 15 import xarray
 16 import rioxarray
 17 import rasterio

 /databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py in import_patch (name, globals, locals, fromlist, level) 
 160 # Import the desired module. If you’re seeing this while debugging a failed import, 
 161 # look at preceding stack frames for relevant error information. 
 --> 162 original_result = python_builtin_import ( name , globals , locals , fromlist , level ) 
 163 
 164 is_root_import = thread_local . _nest_level == 1 

 /databricks/python/lib/python3.8/site-packages/xarray/__init__.py in <module> 
 ----> 1 from xarray import testing , tutorial
 2 from xarray.backends.api import (
 3 load_dataarray , 
 4 load_dataset , 
 5 open_dataarray , 

 /databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py in import_patch (name, globals, locals, fromlist, level) 
 160 # Import the desired module. If you’re seeing this while debugging a failed import, 
 161 # look at preceding stack frames for relevant error information. 
 --> 162 original_result = python_builtin_import ( name , globals , locals , fromlist , level ) 
 163 
 164 is_root_import = thread_local . _nest_level == 1 

 /databricks/python/lib/python3.8/site-packages/xarray/testing.py in <module> 
 5 
 6 import numpy as np
 ----> 7 import pandas as pd
 8 
 9 from xarray . core import duck_array_ops , formatting , utils

 /databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py in import_patch (name, globals, locals, fromlist, level) 
 160 # Import the desired module. If you’re seeing this while debugging a failed import, 
 161 # look at preceding stack frames for relevant error information. 
 --> 162 original_result = python_builtin_import ( name , globals , locals , fromlist , level ) 
 163 
 164 is_root_import = thread_local . _nest_level == 1 

 /databricks/python_shell/dbruntime/PostImportHook.py in load_module (self, fullname) 
 214 def load_module ( self , fullname ) : 
 215 try : 
 --> 216 module = self . loader . load_module ( fullname ) 
 217 notify_module_loaded ( module ) 
 218 except ( ImportError , AttributeError ) : 

 /databricks/python/lib/python3.8/site-packages/pandas/__init__.py in <module> 
 20 
 21 # numpy compat 
 ---> 22 from pandas . compat import is_numpy_dev as _is_numpy_dev # pyright: ignore # noqa:F401 
 23 
 24 try : 

 /databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py in import_patch (name, globals, locals, fromlist, level) 
 160 # Import the desired module. If you’re seeing this while debugging a failed import, 
 161 # look at preceding stack frames for relevant error information. 
 --> 162 original_result = python_builtin_import ( name , globals , locals , fromlist , level ) 
 163 
 164 is_root_import = thread_local . _nest_level == 1 

 /databricks/python/lib/python3.8/site-packages/pandas/compat/__init__.py in <module> 
 16 
 17 from pandas . _typing import F
 ---> 18 from pandas.compat.numpy import (
 19 is_numpy_dev , 
 20 np_version_under1p21 , 

 /databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py in import_patch (name, globals, locals, fromlist, level) 
 160 # Import the desired module. If you’re seeing this while debugging a failed import, 
 161 # look at preceding stack frames for relevant error information. 
 --> 162 original_result = python_builtin_import ( name , globals , locals , fromlist , level ) 
 163 
 164 is_root_import = thread_local . _nest_level == 1 

 /databricks/python/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py in <module> 
 21 
 22 if _nlv < Version ( _min_numpy_ver ) : 
 ---> 23 raise ImportError(
 24 f"this version of pandas is incompatible with numpy < {_

In [0]:
def memory_usage():
    process = psutil.Process(os.getpid())
    print('- - - Current memory usage is:', process.memory_info()[0] / float(2 ** 20))
    
def read_NETCDF(ncPATH):
    '''
    PURPOSE: Read in NetCDF, return an Xarray about with spatial dimension defined, and list of band names (so we know what data is in it)
    INPUTS:
        ncPATH: path to netCDF in Data Lake
    OUTPUTS:
        ds: Xarray
        nc_band: band in data
    '''
    print(ncPATH)
    # Read in arrary
    ds = xarray.open_dataset(ncPATH)
    # Find coordinate names
    dimensions  = [x for x in ds.coords.keys()]
    lat_variable = [x for x in dimensions if "lat" in x][0]
    lon_variable = [x for x in dimensions if "lon" in x][0]
    # Standardize lat and lon names
    ds = ds.rename({lon_variable: 'lon', lat_variable: 'lat'})
    # Set spatial dimenstions and projection
    ds = ds.rio.set_spatial_dims('lon', 'lat')
    ds.rio.crs
    ds.rio.write_crs("epsg:4326", inplace=True)
    # Find name of bands
    nc_bands = list(set([x for x in ds.variables.keys()]) - set(dimensions))
    nc_bands.remove('spatial_ref')
    print(nc_bands)
    return ds

def fillnas(da):
    """Replaces NA values with 0 in data array. Returns data array"""
        # Fill NA's with where statement. fillna functions aren't working great
    da_filled = xarray.where(da.isnull(), 0, da)
    del da
    # reset spatial dimensions
    da_filled = da_filled.rio.set_spatial_dims('lon', 'lat')
    da_filled.rio.crs
    da_filled.rio.write_crs("epsg:4326", inplace=True)
    return da_filled

def resample_xarray(ds, downscale_factor):
    '''
    PURPOSE: Resample NetCDF to smaller size so zonal statistics can be more accurate 
    INPUTS:
        ds: Xarray to downscale
        downscale_factor: 1-dimensional factor to increase size by. 
        Ex: 10 would turn each pixel into 100 smaller, identical pixels (10X10)
    OUTPUTS:
        xds_downscaled: downscaled Xarray
    '''
    # Dfein new dimensions
    new_width = ds.rio.width * downscale_factor
    new_height = ds.rio.height * downscale_factor
    # Run resampling function
    xds_downscaled = ds.rio.reproject(
        ds.rio.crs,
        shape=(new_height, new_width),
        resampling=Resampling.nearest,
    )
    # Rename coordinate dimensions
    xds_downscaled = xds_downscaled.rename({'x': 'lon', 'y': 'lat'})
    return xds_downscaled

def find_demand_path(dtype, scen):
    if dtype != 'livestock':
        years = '1960-2019' if scen == 'historical' else '2000-2100'
        PATH = '/dbfs/mnt/pgb-data-lake/pcrglobwb_input/version_2021-09-16/historical_and_ssp_files/'
        NAME = '{}_water_demand_{}_{}.nc'.format(dtype, scen, years)
        FULL_PATH = PATH + NAME
        
    if dtype == 'livestock':
        PATH = '/dbfs/mnt/pgb-data-lake/pcrglobwb_input/version_2021-09-16/general/'
        NAME = 'livestock_water_demand_05min.nc'
        FULL_PATH = PATH + NAME
    return FULL_PATH
  
def check_regional_extents(area, list_pfs):
    f1, (ax1) = plt.subplots(1, 1,  figsize=(20, 8))
    area['area'].plot(ax = ax1)
    hy6[hy6.index.isin(list_pfs)].boundary.plot(ax = ax1)
    plt.show()
    
def segment_id_list(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
            

    
# ! - - UNIVERSAL DATA - - !
# 1. M folders (ie, regional folders) and path to example regional data
mFolders = ['M' + str(x).zfill(7) for x in range(1, 54)]

# 2. Hydrobasin 6 
geogidlookupPATH = '/dbfs/mnt/pgb-data-lake/aqueduct_dev/aux-boundaries/m_region-pfaf6-lookups/{0}_pfaf6_lookup.csv'.format
shapePATH = '/dbfs/mnt/pgb-data-lake/aqueduct_dev/aux-boundaries/hydro_basin_lv6/aq3_pfaf_basins.shp'
hy6 = gpd.read_file(shapePATH , crs="epsg:4326")
hy6.columns= hy6.columns.str.lower()
project_crs = hy6.crs # WGS84 aka epsg 4326
hy6.set_index('pfaf_id', inplace = True)
geog_id = 'pfaf_id'
# 3. Area
areaPATH = '/dbfs/mnt/pgb-data-lake/aqueduct_dev/aux-boundaries/global_area_5arcmin.nc'
area_band = 'global_cellsize_m2_05min.tif'

# 4.  Output Root ( 0 = resample; 1 =  GCM; 2 = SCEN)
newROOT = '/dbfs/mnt/pgb-data-lake/aqueduct_dev/pcrglobwb_aqueduct_2021/version_2021-09-16/run_202205/zonal_statistics/pfaf6/demand_nonirr_resample_{0}/{1}/'.format
# 4.  Output Name ( 0 = M region; 1 =  PFAF ID)
newNAME = '{0}_{1}.csv'.format

# Main Function

In [0]:
# Clips global data by region, finds withdrawal and consumption for all sectors. Clips by watersheds in region. Saves CSV per PFAF
def run_demand_zonal(m, scen, resample_size, list_pfs):
    
    # In function function
    def clip_by_pfaf_id(p):
        # Step 3.1: Select 1 polygon per loop
        my_geom = hy6.loc[p:p, :]
        # Step 3.2: Clip NetCDF by polygon
        stime = time.time()
        clipped = ds_rs.rio.clip(my_geom.geometry, project_crs, drop=False)
        # Step 3.2 housekeeping
        print('- - - - Clipped NetCDF in {}'.format(time.time()-stime))
        memory_usage()
        # Step 3.3: Sum contents across lat and long
        df_t = clipped.sum(dim = ['lon', 'lat']).to_dataframe()
        # Step 3.3 housekeeping
        del clipped
        # Add geometry ID
        df_t[geog_id] = p
        outPATH = newROOT(resample_size, scen) + newNAME(m, p)
        df_t.to_csv(outPATH)
  
    print("Start {0} zonal stats for region {1}".format(scen, m))
    # - - - STEP 1: READ IN REGION DATA TO GET EXTENTS AND WATERSHEDS
    mstime = time.time()
    # Step 1a: read in example data get get regional extents    
    ds_ex = read_NETCDF(exPATH(m))

    # Step 1b:Grab example data, keep cooridnates
    ds_ex = ds_ex.drop(['evaporation_from_irrigation'])
    print("- Step 1: Found regional data")
    
    # - - - STEP 2: MERGE TO EXTENT OF REGIONAL DATA
    print("- Step 2: Filtering data")
    stime = time.time()
    ds_box = xarray.merge([ds_ex, ds_m], join='left', fill_value=0)
    ds_box = ds_box.fillna(0)
    ds_box = ds_box.where(ds_box.apply(np.isfinite)).fillna(0.0)
    print("- Step 2:Demand data filtered by regional extents in {} seconds".format(time.time() - stime))
    memory_usage()

    # - - - STEP 5: CALCULATE WITHDRAWAL AND DEMAND
    stime = time.time()
    # Step 5a: Calculate demand and consumption
    dtype = 'domestic'
    ds_box = ds_box.assign(gdomww=(xarray.where(ds_box[dtype+'GrossDemand'] > 0, ds_box[dtype+'GrossDemand'] * ds_box['area'], 0 )))
    ds_box = ds_box.assign(gdomwn=(xarray.where(ds_box[dtype+'GrossDemand'] > 0, (ds_box[dtype+'NettoDemand'] / ds_box[dtype+'GrossDemand'] ) * ds_box['gdomww'], 0 )))
    dtype = 'industry'
    ds_box = ds_box.assign(gindww=(xarray.where(ds_box[dtype+'GrossDemand'] > 0, ds_box[dtype+'GrossDemand'] * ds_box['area'], 0 )))
    ds_box = ds_box.assign(gindwn=(xarray.where(ds_box[dtype+'GrossDemand'] > 0, (ds_box[dtype+'NettoDemand'] / ds_box[dtype+'GrossDemand'] ) * ds_box['gindww'], 0 )))
    if scen == "historical":
        dtype = 'livestock'
        ds_box = ds_box.assign(glivww=(xarray.where(ds_box[dtype+'GrossDemand'] > 0, ds_box[dtype+'GrossDemand'] * ds_box['area'], 0 )))
        # Step 5b: Drop OG fields
        ds_box = ds_box.drop(['domesticGrossDemand', 'domesticNettoDemand', 'industryGrossDemand', 'industryNettoDemand', 'livestockGrossDemand','livestockNettoDemand', 'area', area_band])
    else: 
        ds_box = ds_box.drop(['domesticGrossDemand', 'domesticNettoDemand', 'industryGrossDemand', 'industryNettoDemand', 'area', area_band])
    # Step 5c: Reset spatial info (gets reset after where statement)
    ds_box = ds_box.rio.set_spatial_dims('lon', 'lat')
    ds_box.rio.crs
    ds_box.rio.write_crs(ds_ex.rio.crs, inplace = True)
#     del ds_dom
    print("- - Step 5: Demand indicators calculated in {} seconds".format(time.time() - stime))
    memory_usage()
    
    # - - - STEP 6: RESAMPLE DATA
    # Step 6a: Resample data
    stime = time.time()
    print("- Step 4: Resampling data")
    ds_rs = resample_xarray(ds_box, resample_size)
    del ds_box
    ds_rs = ds_rs.chunk({"lon": 100, "lat": 100})
    # Step 6b: Create list of fields
    if scen == "historical":
        fields =  ['gdomww', 'gdomwn','gindww', 'gindwn','glivww']
    else:
        fields =  ['gdomww', 'gdomwn','gindww', 'gindwn']
    # Step cb: Set Fill value for all columns
    for x in fields:
        ds_rs[x].attrs['_FillValue'] = 0.0
    memory_usage()
    print("- - Step 6 complete. Data resampled by {0}x{0} in {1}".format(str(resample_size), time.time() - stime))   
    ds_rs = ds_rs.rio.set_spatial_dims('lon', 'lat')
    ds_rs.rio.crs
    ds_rs.rio.write_crs(ds_ex.rio.crs, inplace = True)
    del ds_ex
    # - - - STEP 7: CREATE DATAFRAME
    # Step 7b: Loop through the watersheds in the region, create a table per watershed
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #
    print(" - - Start Step 7: Start clip loop") 
    memory_usage()
    stime = time.time()
    # Set number of workers
    n_workers = 40
    # Keep track of while loop
    worker_count = 1
    # While catchments remain in objectids, repeat this process. For every new round, lessen the number of workers to prevent memory overload
    run_count = 1
    oid_count = 0
    # Segment catchments by workers for parallel process
    objectids_list = segment_id_list(lst=list_pfs, n= int(n_workers / worker_count))
    # Step 3. Clip and sum by polygon
    df_fs = []
    for oids in tqdm(objectids_list):
        memory_usage()
        Parallel(n_jobs=n_workers)(delayed(clip_by_pfaf_id)(p) for p in oids)
        run_count += 1
        oid_count = len(oids) + oid_count
        print('- - - - - run number', run_count, "\n- - - - - - Remaining catchments:", len(list_pfs) - oid_count)
        gc.collect()
    del ds_rs
    gc.collect()
    endtime = time.time() - mstime
    print('Region {0} done in {1}'.format(m, endtime))

# Run Regions

In [0]:
# USER SELECTIONS!!!
scen_sel = 'ssp585' # Scenario
rs_size = 5 # Resample Size

# READ IN GLOBABL DATA
print("Reading in global data")
ds_area = read_NETCDF(areaPATH)
ds_area = ds_area.assign(area=ds_area[area_band]/1000000.0)
# Step 2a: Create paths to data
DOM_PATH = find_demand_path('domestic', scen_sel[0:4])
IND_PATH = find_demand_path('industry', scen_sel[0:4])
LIV_PATH = find_demand_path('livestock', scen_sel[0:4])
ds_dom = read_NETCDF(DOM_PATH)
ds_ind = read_NETCDF(IND_PATH)

# Reindex area
grid_area = ds_area.reindex_like(ds_dom, method='nearest', tolerance=0.01)
del ds_area
stime = time.time()

# Add Livestock for historical runs (no future livestock)
if scen_sel == 'historical':
    ds_liv = read_NETCDF(LIV_PATH) 
    ds_m = xarray.merge([ds_dom, ds_ind, ds_liv, grid_area])
    # Path to example regional data, will be used to clip global data. Need to specify historic vs. future because of dates
    exPATH = '/dbfs/mnt/pgb-data-lake/pcrglobwb_output1/pcrglobwb_aqueduct_2021/version_2021-09-16/gswp3-w5e5/historical-reference/begin_from_1960/{0}/netcdf/evaporation_from_irrigation_monthTot_output.nc'.format
    del ds_dom, ds_ind, ds_liv, grid_area
else:
    ds_m = xarray.merge([ds_dom, ds_ind, grid_area])  
    # The shape and dates are consistent across scens. We are just trying to clip the global data, and need future dates for future scenarios
    exPATH = '/dbfs/mnt/pgb-data-lake/pcrglobwb_output1/pcrglobwb_aqueduct_2021/version_2021-09-16/gfdl-esm4/ssp370/begin_from_2015/{0}/netcdf/evaporation_from_irrigation_monthTot_output.nc'.format
    del ds_dom, ds_ind, grid_area
# Update date to end of month
s = ds_m['time'].to_series()
s_ceil  = s.dt.to_period('M').dt.to_timestamp('M')
s_ceil
ds_m['time'] = s_ceil


print("Global data merged in {} seconds".format(time.time()-stime ))
print("Looping through regions")

for m in mFolders:
    test_root = os.path.dirname(newROOT(rs_size, scen_sel))
    complete_files = os.listdir(test_root)
    complete_pfs = [int(x.split("_")[1].replace(".csv", "")) for x in complete_files if (m in x)]
    # Read full list of PFs
    df_pf = pd.read_csv(geogidlookupPATH(m))
    all_pfs = list(set(df_pf['pfaf_id'].tolist()))
    # Step 3b: Create list of IDs from lookup table
#     list_pfs = list(set(df_pf['pfaf_id'].tolist()))
    unfinished_pfs = list(set(all_pfs) - set(complete_pfs))
    print("Region {0} has {1} out of {2} watersheds left".format(m, len(unfinished_pfs), len(all_pfs)))
    if len(unfinished_pfs) == 0:
        continue
    else: 
        run_demand_zonal(m = m, scen = scen_sel,  resample_size = rs_size, list_pfs = unfinished_pfs)

Reading in global data

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3196742707517592> in <module> 
 5 # READ IN GLOBABL DATA 
 6 print ( "Reading in global data" ) 
 ----> 7 ds_area = read_NETCDF ( areaPATH ) 
 8 ds_area = ds_area . assign ( area = ds_area [ area_band ] / 1000000.0 ) 
 9 # Step 2a: Create paths to data 

 NameError : name 'read_NETCDF' is not defined